In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
 !pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-b7qpm5hl
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-b7qpm5hl
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 801584cceb559adc54e828ebe9b385c5f53fe70f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download VnCoreNLP pre-trained model
!wget -O VnCoreNLP-1.1.1.jar https://github.com/vncorenlp/VnCoreNLP/blob/master/VnCoreNLP-1.1.1.jar

--2024-07-03 03:53:49--  https://github.com/vncorenlp/VnCoreNLP/blob/master/VnCoreNLP-1.1.1.jar
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar     [ <=>                ] 276.50K  1.51MB/s    in 0.2s    

2024-07-03 03:53:50 (1.51 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [283136]



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%%capture
!pip install datasets
!pip install evaluate
!pip install transformers

In [ ]:
%%capture
# Tải code và cài thư viện transformer

# Tải code
%cd /content
!git clone https://github.com/huggingface/transformers

# Cài thư viện transformer
%cd transformers
!pip install .

In [ ]:
!pip install transformers torch
!pip install vncorenlp
!pip install pandas numpy scikit-learn torch transformers pyspark kafka-python
!pip install findspark
!pip install sodapy

  Using cached pyspark-3.5.1.tar.gz (317.0 MB)
  Preparing metadata (setup.py) ... done
  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=693147ae907af304bc881ee5981b5844e103228477575bc47e009253321b9796
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import datetime
from pyspark.sql import functions as f
from pyspark.conf import SparkConf
from sodapy import Socrata
from pyspark.sql.streaming import DataStreamReader
scala_version = '2.12'
spark_version = '3.5.0'
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.6.1'
]
spark = SparkSession \
.builder \
.appName("Chronos") \
.master("local") \
.config("spark.executor.memory", "16g") \
.config("spark.driver.memory", "16g") \
.config("spark.python.worker.reuse", "true") \
.config("spark.sql.execution.arrow.pyspark.enabled", "true") \
.config("spark.sql.execution.arrow.maxRecordsPerBatch", "16") \
.config("spark.jars.packages", ",".join(packages)) \
.getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

conf=SparkConf()

spark

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, split
from pyspark.sql.types import StructType, StructField, StringType
from torch.utils.data import DataLoader
from transformers import TrainingArguments, Trainer
from pyspark.sql.functions import pandas_udf, PandasUDFType
from sklearn.model_selection import train_test_split
import pyspark
from pyspark.ml.feature import Tokenizer, CountVectorizer, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType

In [ ]:
# Khởi tạo Spark session
spark = SparkSession.builder \
    .appName("NER Prediction with Spark") \
    .getOrCreate()

In [ ]:
# Đọc dữ liệu từ CSV
data = pd.read_csv('/content/output.csv')
data

,Index,Comment,Object,Label,tu_viet_tat,meaning,xoa_khong
0,0,garnacho bảnh quá,garnacho,1.0,NaN,NaN,NaN
1,1,bình an đăng lại kìa,NaN,0.0,NaN,NaN,NaN
2,2,gọi garnacho là gì,garnacho,0.0,NaN,NaN,NaN
3,3,"chỉnh sửa đẹp , ngầu âm thanh chất lượng nhưng...",NaN,0.0,k,không,NaN
4,4,hâm mộ manchester city những vẫn thích united,"manchester city, manchester united",1.0,"mc, mu","manchester city, manchester united",NaN
...,...,...,...,...,...,...,...
5109,5573,romelu lukaku khiến tôi bán xe rồi,romelu lukaku,2.0,lukacặk,romelu lukaku,NaN
5110,5574,bỉ là đội yếu chứ không mạnh như mầy tưởng,bỉ,2.0,NaN,NaN,NaN
5111,5575,đồng tiền lái đôi chân đá trượt bóng luôn khi ...,NaN,2.0,NaN,NaN,NaN
5112,5576,vứt lịch thi đấu mới quan trọng còn nhận định ...,NaN,2.0,NaN,NaN,NaN


In [ ]:
data = data[['Comment', 'Object', 'Label']]
data = data[data['Label'].isin([0, 1, 2])].reset_index(drop=True)
data['Object'] = data['Object'].fillna('').apply(lambda x: x.split(',') if x else [])


In [ ]:
# Chuyển đổi DataFrame từ Pandas sang Spark
spark_df = spark.createDataFrame(data)


# **Huấn luyện model**

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
# Chia dữ liệu thành tập huấn luyện và kiểm tra
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Chuyển đổi DataFrame từ Pandas sang Spark
spark_train_df = spark.createDataFrame(train_df)
spark_test_df = spark.createDataFrame(test_df)

In [ ]:
# Tạo mô hình NER
tokenizer = AutoTokenizer.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/ner-vietnamese-electra-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/411k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/532M [00:00<?, ?B/s]

In [ ]:
#  pipeline
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

# **Dự đoán trên tập dữ liệu kiểm tra**

In [ ]:
# Hàm dự đoán chủ thể từ comment sử dụng NER
def predict_objects(comment):
    ner_results = nlp(comment)
    objects = []
    current_word = ""
    for result in ner_results:
        word = result['word']
        if word.startswith("##"):
            current_word += word[2:]
        else:
            if current_word:
                objects.append(current_word)
            current_word = word
    if current_word:
        objects.append(current_word)
    return objects

In [ ]:
# Tạo UDF để sử dụng hàm dự đoán trong Spark
predict_objects_udf = udf(predict_objects, ArrayType(StringType()))

In [ ]:
# Áp dụng hàm dự đoán vào DataFrame
spark_df = spark_df.withColumn("Predicted_Objects", predict_objects_udf(spark_df["Comment"]))

In [ ]:
# Hiển thị kết quả
spark_df.select("Comment", "Predicted_Objects").show(truncate=False)


+-----------------------------------------------------------------+----------------------------+
|Comment                                                          |Predicted_Objects           |
+-----------------------------------------------------------------+----------------------------+
|garnacho bảnh quá                                                |[garnacho]                  |
|bình an đăng lại kìa                                             |[]                          |
|gọi garnacho là gì                                               |[]                          |
|chỉnh sửa đẹp , ngầu âm thanh chất lượng nhưng không dám đăng lại|[]                          |
|hâm mộ manchester city những vẫn thích united                    |[manchester, city, united]  |
|anh lực antony quá dữ                                            |[]                          |
|garnacho đẹp ác lại còn bảnh                                     |[garnacho]                  |
|gọi garnacho là ghế vì nó khô

# **Đánh giá mô hình**

In [ ]:
from pyspark.sql import functions as F

In [ ]:
# Chuyển đổi các đối tượng từ dạng danh sách sang chuỗi
spark_df = spark_df.withColumn("Object_str", F.concat_ws(",", col("Object")))
spark_df = spark_df.withColumn("Predicted_Objects_str", F.concat_ws(",", col("Predicted_Objects")))

In [ ]:
# Tính toán True Positives, False Positives, và False Negatives
true_positives = spark_df.withColumn("tp", size(array_except(col("Object"), col("Predicted_Objects"))))
false_positives = spark_df.withColumn("fp", size(array_except(col("Predicted_Objects"), col("Object"))))
false_negatives = spark_df.withColumn("fn", size(array_except(col("Object"), col("Predicted_Objects"))))


In [ ]:
# Tính toán số lượng các giá trị tp, fp, fn
tp_sum = true_positives.select(F.sum(col("tp")).alias("tp_sum")).collect()[0]["tp_sum"]
fp_sum = false_positives.select(F.sum(col("fp")).alias("fp_sum")).collect()[0]["fp_sum"]
fn_sum = false_negatives.select(F.sum(col("fn")).alias("fn_sum")).collect()[0]["fn_sum"]

# Tính độ chính xác và F1 score
accuracy = tp_sum / (tp_sum + fp_sum + fn_sum) if tp_sum + fp_sum + fn_sum != 0 else 0
f1_score = 2 * tp_sum / (2 * tp_sum + fp_sum + fn_sum) if 2 * tp_sum + fp_sum + fn_sum != 0 else 0

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1_score}")


Accuracy: 0.3635719347980156
F1 Score: 0.5332640332640333


In [ ]:
# Lưu kết quả vào file CSV
output_path = '/content/predicted_output.csv'
spark_df.toPandas().to_csv(output_path, index=False)
